In [1]:
import json
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import chain
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Read data

In [2]:
# # Load the pickled list
# with open("/Utilisateurs/umushtaq/emotion_analysis_comics/incontext_learning/results/icl_llama-3-8b-Instruct-bnb-4bit/results_10.pickle", 'rb') as f:
    
#     results = pickle.load(f)

In [3]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/meld_FT/saved_models/meld_Qwen2.5-72B-Instruct-bnb-4bit/meld_results_2.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [4]:
#all_labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [5]:
grounds = results['ground_truths']
predictions = results['predictions']

In [6]:
len(grounds), len(predictions)

(2610, 2610)

In [7]:
grounds

['{"emotion_class": ["surprise"]}',
 '{"emotion_class": ["anger"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["sadness"]}',
 '{"emotion_class": ["surprise"]}',
 '{"emotion_class": ["anger"]}',
 '{"emotion_class": ["anger"]}',
 '{"emotion_class": ["anger"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["joy"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["fear"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["anger"]}',
 '{"emotion_class": ["disgust"]}',
 '{"emotion_class": ["neutral"]}',
 '{"emotion_class": ["surprise"]

In [8]:
predictions

[{'role': 'assistant', 'content': '{"emotion_class": ["surprise"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["joy"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["joy"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["joy"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["anger"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["anger"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["surprise"]}'},
 {'role': 'assistant', 'content': '{"emotion_class": ["neutral"]}'},
 {'role': 'assistant', 'content': '{"emotion_cla

In [9]:
grounds = [json.loads(x)["emotion_class"] for x in grounds]

In [10]:
predictions = [json.loads(x["content"]) for x in predictions]

In [11]:
predictions = [x['emotion_class'] for x in predictions]

In [12]:
len(grounds), len(predictions)

(2610, 2610)

In [13]:
predictions

[['surprise'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['anger'],
 ['anger'],
 ['neutral'],
 ['surprise'],
 ['neutral'],
 ['neutral'],
 ['anger'],
 ['neutral'],
 ['neutral'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['sadness'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['disgust'],
 ['joy'],
 ['joy'],
 ['neutral'],
 ['joy'],
 ['surprise'],
 ['surprise'],
 ['fear'],
 ['disgust'],
 ['anger'],
 ['neutral'],
 ['disgust'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['surprise'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['surprise'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['surprise'],
 ['joy'],
 ['anger'],
 ['sadness'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['neutral'],
 ['joy'],
 ['neutral'],
 ['neutral'],
 ['surprise'],
 ['surprise'],
 ['neutral'],
 ['surprise'],
 ['anger'],
 ['fear']

In [14]:
grounds_l = [len(element) for element in grounds]
predictions_l = [len(element) for element in predictions]

In [15]:
counts_grounds = Counter(grounds_l)
counts_predictions = Counter(predictions_l)

In [16]:
counts_grounds

Counter({1: 2610})

In [17]:
counts_predictions

Counter({1: 2610})

In [18]:
#true_matrix, predicted_matrix = post_process_icl(grounds, predictions)

In [19]:
grounds = [elem for sublist in grounds for elem in sublist]
predictions = [elem for sublist in predictions for elem in sublist]


In [20]:
len(grounds), len(predictions)

(2610, 2610)

## Classification Report

In [21]:
def custom_classification_report(y_true, y_pred, exclude_class=None):
    """
    Generate a classification report excluding a specific class from weighted F1-score calculation.
    
    Parameters:
    y_true (list or array): True labels.
    y_pred (list or array): Predicted labels.
    exclude_class (int or str): Class to exclude from weighted F1-score calculation.
    
    Returns:
    dict: Classification report as a dictionary.
    """
    report = classification_report(y_true, y_pred, output_dict=True)
    
    if exclude_class is not None and str(exclude_class) in report:
        # Remove the specified class
        report.pop(str(exclude_class), None)
        
        # Recalculate weighted F1-score manually
        support = np.array([report[label]['support'] for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')])
        f1_scores = np.array([report[label]['f1-score'] for label in report if label not in ('accuracy', 'macro avg', 'weighted avg')])
        
        if support.sum() > 0:
            weighted_f1 = np.sum(f1_scores * support) / support.sum()
        else:
            weighted_f1 = 0.0
        
        report['weighted avg']['f1-score'] = weighted_f1
    
    return report

In [23]:
report = custom_classification_report(grounds, predictions, exclude_class="neutral")

In [25]:
report

{'anger': {'precision': 0.587248322147651,
  'recall': 0.5072463768115942,
  'f1-score': 0.5443234836702955,
  'support': 345.0},
 'disgust': {'precision': 0.5,
  'recall': 0.4117647058823529,
  'f1-score': 0.45161290322580644,
  'support': 68.0},
 'fear': {'precision': 0.4358974358974359,
  'recall': 0.34,
  'f1-score': 0.38202247191011235,
  'support': 50.0},
 'joy': {'precision': 0.6564885496183206,
  'recall': 0.6417910447761194,
  'f1-score': 0.6490566037735849,
  'support': 402.0},
 'sadness': {'precision': 0.5726495726495726,
  'recall': 0.32211538461538464,
  'f1-score': 0.4123076923076923,
  'support': 208.0},
 'surprise': {'precision': 0.5476923076923077,
  'recall': 0.6334519572953736,
  'f1-score': 0.5874587458745875,
  'support': 281.0},
 'accuracy': 0.6869731800766283,
 'macro avg': {'precision': 0.5820309170791098,
  'recall': 0.5297543282009004,
  'f1-score': 0.5482860746350247,
  'support': 2610.0},
 'weighted avg': {'precision': 0.6773035709884739,
  'recall': 0.68697

In [75]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

       anger      0.587     0.507     0.544       345
     disgust      0.500     0.412     0.452        68
        fear      0.436     0.340     0.382        50
         joy      0.656     0.642     0.649       402
     neutral      0.774     0.852     0.811      1256
     sadness      0.573     0.322     0.412       208
    surprise      0.548     0.633     0.587       281

    accuracy                          0.687      2610
   macro avg      0.582     0.530     0.548      2610
weighted avg      0.677     0.687     0.677      2610

